In [ ]:
# Path setup to resolve package imports (dynamic project root)
import sys, os

# Derive project root from current notebook directory: <project>/notebook
notebook_dir = os.getcwd()
project_root = os.path.dirname(notebook_dir)

if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [ ]:
from src.agents.tools.db_tools import DBToolManager
# Test tool manager
tool_mgr = DBToolManager()
tools = tool_mgr.get_tools()
print(f"✅ {len(tools)} tools ready")
print("Tool names:", [t.name for t in tools])


In [ ]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
from src.agents.prompt import system_prompt
from langchain_core.messages import HumanMessage

checkpointer = MemorySaver()

gptLLM = ChatOpenAI(
    model_name="gpt-4",temperature=0
)


agent = create_agent(
    model=gptLLM,
    tools=tools,
    system_prompt=system_prompt,
    checkpointer=checkpointer
)

def chat_with_steps(user_id, question):
    tool_mgr.clear_steps()
    config = {"configurable": {"thread_id": user_id}}
    
    result = agent.invoke(
        {"messages": [HumanMessage(content=question)]},
        config
    )
    
    final_msg = result["messages"][-1].content
    steps = tool_mgr.get_step_summary()
    
    return f"""
🎯 **ANSWER**
{final_msg}

{steps}

**Total: {len(tool_mgr.steps)} steps**
    """.strip()


In [ ]:
print("🧪 Testing step tracing...")
print("="*60)

response = chat_with_steps("test_user", "employees in Sales department HumanResources")
print(response)
